### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[10. 10. 10. 10.]
 [10.  4.  9. 10.]
 [ 8.  9. 10.  5.]
 [ 9.  8. 10. 10.]
 [10.  5.  9.  9.]
 [ 6.  4. 10.  6.]
 [ 9.  8. 10.  9.]
 [10.  5.  9.  8.]
 [ 7.  8. 10.  8.]
 [ 9.  5.  9.  7.]
 [ 9.  8. 10.  8.]
 [ 9. 10. 10.  9.]
 [10.  9. 10.  8.]
 [ 5.  8.  5.  8.]
 [10.  8. 10. 10.]
 [ 9.  9. 10. 10.]
 [ 9.  8.  8.  8.]
 [10.  8.  1. 10.]
 [ 5.  6. 10. 10.]
 [ 8.  7. 10.  7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [2]:
np.random.seed(0)

In [6]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
         b - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0] # number of rows in the matrix
    n_movies = ratings_mat.shape[1]  # number of movies in the matrix
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat)) # total number of ratings in the matrix

    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users,latent_features) # user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies)# movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0

    # header for running results
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")

    # for each iteration
    for iteration in range(iters):
        # update our sse
        old_sse = sse_accum
        sse_accum = 0

        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                # if the rating exists
                if ratings_mat[i,j] > 0:
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    error = ratings_mat[i,j] - np.dot(user_mat[i,:], movie_mat[:,j])
                    # Keep track of the total sum of squared errors for the matrix
                    sse_accum += error ** 2
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i,k] += learning_rate * 2 * error * movie_mat[k,j]
                        movie_mat[k,j] += learning_rate * 2 * error * user_mat[i,k]
        # print results for iteration
        print("%d \t\t %f"%(iteration+1, sse_accum/ num_ratings))
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [7]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10)# use your function with 4 latent features, lr of 0.005 and 10 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 41.945046
2 		 13.411718
3 		 3.826587
4 		 2.810052
5 		 2.687706
6 		 2.636433
7 		 2.596918
8 		 2.557880
9 		 2.516174
10 		 2.470572


In [8]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 9.97366597  8.72703972 10.91797861 10.02931495]
 [ 9.06346061  7.10782838  9.25409011  8.58617772]
 [ 7.87842697  6.79382979  8.2128743   7.87569555]
 [ 9.70038801  8.27247723  9.83733721  9.65977911]
 [ 8.63525487  7.15759512  9.18598502  8.45060342]
 [ 6.86556439  5.91491223  7.28601638  6.83196394]
 [ 9.2380342   7.89145665  9.78920763  9.17407688]
 [ 8.23824694  7.22794911  8.47268461  8.26988456]
 [ 8.39304777  7.45873057  8.9623173   8.46705646]
 [ 7.98054818  6.28156993  8.19116952  7.63779498]
 [ 8.80740426  7.78523518  9.31739982  8.91075897]
 [ 9.62824735  8.52286916  9.92416576  9.60771395]
 [ 9.33865865  8.37463333  9.39622306  9.37835679]
 [ 6.50754751  6.28770097  6.48367658  6.81434882]
 [10.01691269  8.41708865 10.00140517  9.83355479]
 [ 9.42826791  8.85454161 10.14737018  9.80014689]
 [ 8.45192842  7.13450983  8.68447992  8.2881513 ]
 [ 6.88909219  6.74071125  7.02862783  7.24623746]
 [ 8.10585896  7.47172229  8.8925103   8.34445652]
 [ 8.21817724  6.74268637  8.82

**Write your summary here.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [9]:
user_mat, movie_mat =FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250) #use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 42.134401
2 		 14.302274
3 		 3.992949
4 		 2.806334
5 		 2.691264
6 		 2.646094
7 		 2.607877
8 		 2.568432
9 		 2.525467
10 		 2.477866
11 		 2.424779
12 		 2.365448
13 		 2.299205
14 		 2.225511
15 		 2.144013
16 		 2.054619
17 		 1.957572
18 		 1.853521
19 		 1.743572
20 		 1.629304
21 		 1.512727
22 		 1.396171
23 		 1.282117
24 		 1.172978
25 		 1.070869
26 		 0.977418
27 		 0.893642
28 		 0.819918
29 		 0.756044
30 		 0.701359
31 		 0.654903
32 		 0.615556
33 		 0.582171
34 		 0.553655
35 		 0.529024
36 		 0.507422
37 		 0.488130
38 		 0.470552
39 		 0.454201
40 		 0.438686
41 		 0.423693
42 		 0.408973
43 		 0.394333
44 		 0.379626
45 		 0.364744
46 		 0.349617
47 		 0.334203
48 		 0.318492
49 		 0.302497
50 		 0.286260
51 		 0.269838
52 		 0.253313
53 		 0.236777
54 		 0.220337
55 		 0.204106
56 		 0.188197
57 		 0.172726
58 		 0.157798
59 		 0.143511
60 		 0.129946
61 		 0.117169
62 		 0.105230
63 		 0.094158
64 		

In [10]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[10.00000544 10.00000472 10.0000019   9.99999288]
 [10.00000023  4.00000023  9.00000006  9.99999959]
 [ 8.00000519  9.00000438 10.0000017   4.9999932 ]
 [ 8.99999853  7.99999873  9.9999994  10.00000172]
 [10.00001055  5.0000087   9.00000338  8.99998646]
 [ 5.99999184  3.99999312  9.99999713  6.00001025]
 [ 8.99999534  7.99999612  9.99999836  9.00000575]
 [10.00000071  5.00000062  9.00000019  7.99999895]
 [ 7.00001157  8.00000956 10.00000374  7.99998518]
 [ 9.00000726  5.0000058   9.00000214  6.99999066]
 [ 8.99999758  7.99999766  9.99999882  8.00000294]
 [ 9.00000235 10.00000162 10.00000042  8.99999689]
 [ 9.99999482  8.99999545  9.99999796  8.0000064 ]
 [ 5.00000087  8.00000059  5.00000013  7.99999881]
 [ 9.99999469  7.99999545  9.99999804 10.0000066 ]
 [ 9.00000203  9.00000168 10.00000059  9.99999724]
 [ 9.00000014  8.00000011  7.99999995  7.99999964]
 [ 9.99999764  7.99999796  0.9999991  10.00000291]
 [ 5.00000039  6.00000028 10.00000005  9.99999942]
 [ 7.99999201  6.99999347  9.99

**Write your summary here.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [11]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[nan, 10., 10., 10.],
        [10.,  4.,  9., 10.],
        [ 8.,  9., 10.,  5.],
        [ 9.,  8., 10., 10.],
        [10.,  5.,  9.,  9.],
        [ 6.,  4., 10.,  6.],
        [ 9.,  8., 10.,  9.],
        [10.,  5.,  9.,  8.],
        [ 7.,  8., 10.,  8.],
        [ 9.,  5.,  9.,  7.],
        [ 9.,  8., 10.,  8.],
        [ 9., 10., 10.,  9.],
        [10.,  9., 10.,  8.],
        [ 5.,  8.,  5.,  8.],
        [10.,  8., 10., 10.],
        [ 9.,  9., 10., 10.],
        [ 9.,  8.,  8.,  8.],
        [10.,  8.,  1., 10.],
        [ 5.,  6., 10., 10.],
        [ 8.,  7., 10.,  7.]])

In [12]:
# run SVD on the matrix with the missing value
user_mat, movie_mat =FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250) #use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 42.431460
2 		 14.901839
3 		 4.308666
4 		 2.961857
5 		 2.811298
6 		 2.771782
7 		 2.748853
8 		 2.727641
9 		 2.705207
10 		 2.680848
11 		 2.654205
12 		 2.624941
13 		 2.592720
14 		 2.557219
15 		 2.518152
16 		 2.475286
17 		 2.428453
18 		 2.377557
19 		 2.322580
20 		 2.263579
21 		 2.200679
22 		 2.134060
23 		 2.063934
24 		 1.990527
25 		 1.914066
26 		 1.834768
27 		 1.752852
28 		 1.668560
29 		 1.582190
30 		 1.494144
31 		 1.404962
32 		 1.315345
33 		 1.226154
34 		 1.138369
35 		 1.053023
36 		 0.971116
37 		 0.893515
38 		 0.820878
39 		 0.753600
40 		 0.691801
41 		 0.635350
42 		 0.583915
43 		 0.537028
44 		 0.494155
45 		 0.454755
46 		 0.418322
47 		 0.384418
48 		 0.352682
49 		 0.322839
50 		 0.294692
51 		 0.268110
52 		 0.243019
53 		 0.219385
54 		 0.197202
55 		 0.176477
56 		 0.157223
57 		 0.139447
58 		 0.123147
59 		 0.108302
60 		 0.094878
61 		 0.082823
62 		 0.072070
63 		 0.062540
64 		

In [13]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 9.456818684075545:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [15]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=0.005, iters=20)#fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 23.723376
2 		 10.785849
3 		 7.391817
4 		 5.699138
5 		 4.630904
6 		 3.884768
7 		 3.331569
8 		 2.904951
9 		 2.566672
10 		 2.292766
11 		 2.067299
12 		 1.879245
13 		 1.720731
14 		 1.585965
15 		 1.470554
16 		 1.371078
17 		 1.284824
18 		 1.209613
19 		 1.143678
20 		 1.085580


`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [16]:
# Replace each of the comments below with the correct values
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))# How many actual ratings exist in first_1000_users
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()


ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings# How many ratings did we make for user-movie pairs that didn't actually have ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [17]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
